In [1]:
import json

In [5]:
PATH_FILE_DATA = 'C:/Users/ADMIN/Desktop/DATN/Extract_Information/data_crawl/data/data_crawl.jsonl'
PATH_FILE_TEMPLATE = 'C:/Users/ADMIN/Desktop/DATN/Extract_Information/tasks/task_template.json'

In [3]:
category_attribute_2_human_msg = {
    'Laptops': 'Acer Aspire S7 13.3-Inch Touchscreen Ultrabook (Intel Core i5-4200U Processor up to 2.6GHz, 8GB DDR3 Memory, 256GB SSD, FHD 1080P Touchscreen Display, Backlit Keyboard, 802.11AC WiFi, Windows 8.1, 0.5 inch thin and 2.9 Lbs) Crystal White',
}

category_attribute_2_system_msg = {
    'Laptops': 'Brand: Acer \n Model: Aspire S7 \n Screen Size: 13.3 inches \n Screen Type: Touchscreen \n Processor Brand: Intel Core i5-4200U \n Processor Speed: 2.6GHz \n Memory: 8GB DDR3 \n Storage: 256GB SSD \n Resolution: FHD 1080P \n Keyboard: Backlit\n Wireless Connectivity: 802.11AC WiFi \n Operating System: Windows 8.1 \n Color: Crystal White \n Thickness: 0.5 inch \n Weight: 2.9 lbs',
}

# Update Task Dict
def update_task_dict_from_file(file_name, task_dict):
    with open(file_name, 'r') as f:
        for line in f:
            record = json.loads(line)
            example = {}
            example['human_0'] = category_attribute_2_human_msg['Laptops']
            example['system_0'] = category_attribute_2_system_msg['Laptops']
            example['input'] = record['title']

            task_dict['examples'].append(example)
    return task_dict

In [6]:
# Load task template
with open(PATH_FILE_TEMPLATE, 'r') as f:
    task_dict = json.load(f)

task_dict['task_prefix'] = "Extract all important attribute name : attribute value pairs mentioned in the product title below. All parts of the product title should be assigned to an attribute. \n Desired format: <attribute name>: <attribute value> \n <attribute name>: <attribute value>"

# Generate examples
task_dict = update_task_dict_from_file(PATH_FILE_DATA, task_dict)

with open('task_data_crawl.json', 'w', encoding='utf-8') as f:
    json.dump(task_dict, f, indent=4)

## Evaluate Task

In [7]:
from dotenv import load_dotenv

# print the dictionary
print(task_dict)

# Load OPENAI_API_KEY from .env file
load_dotenv()

{'name': 'product_information_extraction', 'description': 'Product Information Extraction deals with the extraction of product information based on an attribute from product title & product description', 'examples': [{'human_0': 'Acer Aspire S7 13.3-Inch Touchscreen Ultrabook (Intel Core i5-4200U Processor up to 2.6GHz, 8GB DDR3 Memory, 256GB SSD, FHD 1080P Touchscreen Display, Backlit Keyboard, 802.11AC WiFi, Windows 8.1, 0.5 inch thin and 2.9 Lbs) Crystal White', 'system_0': 'Brand: Acer \n Model: Aspire S7 \n Screen Size: 13.3 inches \n Screen Type: Touchscreen \n Processor Brand: Intel Core i5-4200U \n Processor Speed: 2.6GHz \n Memory: 8GB DDR3 \n Storage: 256GB SSD \n Resolution: FHD 1080P \n Keyboard: Backlit\n Wireless Connectivity: 802.11AC WiFi \n Operating System: Windows 8.1 \n Color: Crystal White \n Thickness: 0.5 inch \n Weight: 2.9 lbs', 'input': 'HP 15s fq5229TU i3 1215U (8U237PA) (Intel UHD, 15.6 inches, 1.2GHz)', 'Screen Size': '15.6 inches', 'Processor Brand': 'Inte

True

In [8]:
# Initialize processing
from tqdm import tqdm
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI

from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# Put task prefix into system message.
prompt_list = []
system_message_prompt = SystemMessagePromptTemplate.from_template(task_dict['task_prefix'])
prompt_list.append(system_message_prompt)

# Prepare 1-shot  
human_example_template="{human_msg_0}"
human_example_message_prompt = HumanMessagePromptTemplate.from_template(human_example_template)
prompt_list.append(human_example_message_prompt)

ai_example_template="{ai_msg_0}"
ai_example_message_prompt = AIMessagePromptTemplate.from_template(ai_example_template)
prompt_list.append(ai_example_message_prompt)

system_message_prompt_2 = SystemMessagePromptTemplate.from_template(task_dict['task_prefix'])
prompt_list.append(system_message_prompt_2)

# Add 
human_template="{input_string}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
prompt_list.append(human_message_prompt)

chat_prompt = ChatPromptTemplate.from_messages(prompt_list)

gpt_3_turbo = ChatOpenAI(model_name='gpt-3.5-turbo-0125', temperature=0)

llm_chain = LLMChain(
    prompt=chat_prompt,
    llm=gpt_3_turbo
)

# Save model
task_dict['model'] = llm_chain.llm.model_name

In [9]:
def parse_gpt_response_to_dict(response):
    # Split Response by new line
    response_parts = response.split('\n')
    response_parts = [part.strip() for part in response_parts if part.strip() != '']

    # Try to parse line as a JSON object - 2nd attempt
    response_dict = {}

    i = 0

    while i < len(response_parts):
        parsed_response_part = response_parts[i].replace('Human:', '')\
                            .replace('AI:', '').replace('System:', '').strip()
        if ':' in parsed_response_part:
            parsed_response_part = parsed_response_part.split(':')

        if len(parsed_response_part) == 2 and parsed_response_part[1] == " {":
            temp = ""
            i = i + 1
            while response_parts[i] != "}":
                temp = temp + response_parts[i]+" "
                i = i + 1
            response_dict[parsed_response_part[0].replace('"', '').replace("'", "").strip()] = temp.replace('"', '').replace("'", "").replace("[","").replace("]","").strip().rstrip(',!')
        elif len(parsed_response_part) == 2 and isinstance(parsed_response_part[0], str) and isinstance(parsed_response_part[1], list):
            parse_str = ", ".join(parsed_response_part[1])
            parsed_response_part[1] = parse_str
            response_dict[parsed_response_part[0].replace('"', '').replace("'", "").strip()] = parsed_response_part[1].replace('"', '').replace("'", "").replace("[","").replace("]","").strip().rstrip(',!')
        elif len(parsed_response_part) == 2 and isinstance(parsed_response_part[0], str) and isinstance(parsed_response_part[1], str):
            response_dict[parsed_response_part[0].replace('"', '').replace("'", "").strip()] = parsed_response_part[1].replace('"', '').replace("'", "").replace("[","").replace("]","").strip().rstrip(',!')
        i = i + 1

    return response_dict

In [10]:
def select_and_run_llm(human_0, system_0, input):
        pred = None
        if len(input) == 0:
            # No input text provided.
            return pred
        
        response = llm_chain.run({'human_msg_0': human_0, 'ai_msg_0': system_0, 'input_string': input})
        try:
            pred = parse_gpt_response_to_dict(response)
            print(response)
        except Exception as e:
            print(e)
            print('Response: ')
            print(response)
        return pred

In [11]:
# General task prefix
from tqdm import tqdm
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    preds = [select_and_run_llm(example['human_0'], example['system_0'], example['input']) for example in tqdm(task_dict['examples'])]
    
    task_dict['total_tokens'] = cb.total_tokens
    print(f"Total Tokens: {cb.total_tokens}")

  0%|          | 0/447 [00:00<?, ?it/s]

  0%|          | 1/447 [00:02<17:46,  2.39s/it]

Brand: HP 
 Model: 15s fq5229TU 
 Processor: Intel Core i3 1215U 
 Model Number: 8U237PA 
 Graphics: Intel UHD 
 Screen Size: 15.6 inches 
 Processor Speed: 1.2GHz


  0%|          | 2/447 [00:04<15:39,  2.11s/it]

Brand: HP 
 Model: 240 G9 
 Processor: Intel Core i3-1215U 
 Graphics: Intel UHD 
 Screen Size: 14 inches 
 Processor Speed: 1.2GHz 
 Model Number: 6L1X7PA


...

Brand: Lenovo 
 Model: Ideapad Flex 5 14ALC7 R7 5700U (82R900ECVN) 
 Processor: Ryzen 7 
 Screen Size: 14 inches


100%|█████████▉| 446/447 [11:59<00:01,  1.42s/it]

Brand: Lenovo 
 Model: IdeaPad Slim 3 14IAH8 
 Model Number: 83EQ0009VN 
 Processor: Core i5-12450H 
 Screen Size: 14 inches


100%|██████████| 447/447 [12:01<00:00,  1.61s/it]

Brand: Lenovo 
 Model: Ideapad Flex 5 14ALC7 R5 5500U (82R900EDVN) 
 Processor: Ryzen 5 
 Screen Size: 14 inches
Total Tokens: 187000


In [12]:
print(preds)

[{'Brand': 'HP', 'Model': '15s fq5229TU', 'Processor': 'Intel Core i3 1215U', 'Model Number': '8U237PA', 'Graphics': 'Intel UHD', 'Screen Size': '15.6 inches', 'Processor Speed': '1.2GHz'}, {'Brand': 'HP', 'Model': '240 G9', 'Processor': 'Intel Core i3-1215U', 'Graphics': 'Intel UHD', 'Screen Size': '14 inches', 'Processor Speed': '1.2GHz', 'Model Number': '6L1X7PA'}, {'Brand': 'HP', 'Series': 'Pavilion', 'Model': '15 eg2081TU', 'Processor': 'Intel Core i5 1240P', 'Graphics': 'Intel Iris Xe', 'Screen Size': '15.6 inches', 'Processor Speed': '1.7GHz'}, {'Brand': 'HP', 'Model': '245 G10 R5 7520U (8F155PA)', 'Processor': 'AMD Ryzen 5 7520U', 'Graphics': 'Radeon', 'Screen Size': '14 inches', 'Processor Speed': '2.8GHz'}, {'Brand': 'HP', 'Series': 'Pavilion 14 dv2073TU', 'Model': 'i5 1235U (7C0P2PA)', 'Processor Brand': 'Intel', 'Processor Model': 'i5 1235U', 'Graphics': 'Intel Iris Xe', 'Screen Size': '14 inches', 'Processor Speed': '1.3GHz'}, {'Brand': 'HP', 'Model': '240 G9', 'Processor'

In [13]:
# Tên tệp JSON bạn muốn lưu
file_name = "result_open.json"

# Ghi từ điển vào tệp JSON
with open(file_name, 'w') as json_file:
    json.dump(preds, json_file)

In [14]:
preds=[]
with open(file_name, 'r') as f:
    task_results = json.load(f)
    for task_result in task_results:
        preds.append(task_result)

data=[]
for pred, example in zip(preds, task_dict['examples']):
    extract_product={}
    extract_product['title'] = example['input']
    if 'Screen Size' in pred:
        extract_product['Screen Size'] = pred['Screen Size']
    else:
        extract_product['Screen Size'] = 'I do not know.'
    
    if 'Processor Brand' in pred:    
        extract_product['Processor Brand'] = pred['Processor Brand']
    elif 'Processor' in pred: 
        extract_product['Processor Brand'] = pred['Processor']
    else:    
        extract_product['Processor Brand'] = 'I do not know.'
        
    if 'Processor Speed' in pred:
        extract_product['Processor Speed'] = pred['Processor Speed']
    else:
        extract_product['Processor Speed'] = 'I do not know.'
    data.append(extract_product)

In [15]:
with open('C:/Users/ADMIN/Desktop/DATN/Extract_Information/data_crawl/results/results_open_datacrawl.json', 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file)

In [16]:
import pandas as pd
# Tạo DataFrame từ danh sách
df = pd.DataFrame(data)

In [17]:
df.to_csv('extract_gpt.csv', index=False, encoding='utf-8')

In [2]:
import json
with open('C:/Users/ADMIN/Desktop/DATN/Extract_Information/data_crawl/results/results_open_datacrawl.json', 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)
  
data_compare = []  
with open('C:/Users/ADMIN/Desktop/DATN/Extract_Information/data_crawl/data/data_crawl.jsonl', 'r') as f:
    for line in f:
        product = json.loads(line)  
        extract_product = {}
        extract_product['title'] = product['title']
        extract_product['Screen Size'] = product['Screen Size']
        extract_product['Processor Brand'] = product['Processor Brand']
        extract_product['Processor Speed'] = product['Processor Speed']
        data_compare.append(extract_product)

print(data)
print(data_compare)

[{'title': 'HP 15s fq5229TU i3 1215U (8U237PA) (Intel UHD, 15.6 inches, 1.2GHz)', 'Screen Size': '15.6 inches', 'Processor Brand': 'Intel Core i3 1215U', 'Processor Speed': '1.2GHz'}, {'title': 'HP 240 G9 i3 1215U (6L1X7PA) (Intel UHD, 14 inches, 1.2GHz)', 'Screen Size': '14 inches', 'Processor Brand': 'Intel Core i3-1215U', 'Processor Speed': '1.2GHz'}, {'title': 'HP Pavilion 15 eg2081TU i5 1240P (7C0Q4PA) (Intel Iris Xe, 15.6 inches, 1.7GHz)', 'Screen Size': '15.6 inches', 'Processor Brand': 'Intel Core i5 1240P', 'Processor Speed': '1.7GHz'}, {'title': 'HP 245 G10 R5 7520U (8F155PA) (Radeon, 14 inches, 2.8GHz)', 'Screen Size': '14 inches', 'Processor Brand': 'AMD Ryzen 5 7520U', 'Processor Speed': '2.8GHz'}, {'title': 'HP Pavilion 14 dv2073TU i5 1235U (7C0P2PA) (Intel Iris Xe, 14 inches, 1.3GHz)', 'Screen Size': '14 inches', 'Processor Brand': 'Intel', 'Processor Speed': '1.3GHz'}, {'title': 'HP 240 G9 i5 1235U (6L1Y2PA) (Intel UHD, 14 inches, 1.3GHz)', 'Screen Size': '14 inches', '

In [3]:
eval_nn = 0 # the model can predict No value (I do not know.) when ground truth is No attribute value
eval_nv = 0 # some incorrect Value when ground truth is No attribute value
eval_vn = 0 # the model can predict No value (I do not know.) when the ground truth has attribute Values
eval_vc = 0 # Correct values when the ground truth has attribute Values (TP)
eval_vw = 0 # Wrong values when the ground truth has attribute Values (FN-FP)

for dt, dt_compare in zip(data,data_compare):
    if dt_compare['Screen Size'] == "I do not know.":
        if dt_compare['Screen Size'] == dt['Screen Size']:
            eval_nn += 1
        else:
            eval_nv += 1
    else:   
        if dt_compare['Screen Size'] == dt['Screen Size']:
            eval_vc += 1
        elif dt['Screen Size'] == "I do not know.":
            eval_vn += 1
        else:
            eval_vw += 1
            
    if dt_compare['Processor Brand'] == "I do not know.":
        if dt_compare['Processor Brand'] == dt['Processor Brand']:
            eval_nn += 1
        else:
            eval_nv += 1
    else:   
        if dt_compare['Processor Brand'] == dt['Processor Brand']:
            eval_vc += 1
        elif dt['Processor Brand'] == "I do not know.":
            eval_vn += 1
        else:
            eval_vw += 1
            
    if dt_compare['Processor Speed'] == "I do not know.":
        if dt_compare['Processor Speed'] == dt['Processor Speed']:
            eval_nn += 1
        else:
            eval_nv += 1
    else:   
        if dt_compare['Processor Speed'] == dt['Processor Speed']:
            eval_vc += 1
        elif dt['Processor Speed'] == "I do not know.":
            eval_vn += 1
        else:
            eval_vw += 1

precision = round((eval_vc / (eval_nv + eval_vc + eval_vw))*100, 2) if (eval_nv + eval_vc + eval_vw) > 0 else 0 
recall = round((eval_vc / (eval_vn + eval_vc + eval_vw))*100, 2) if (eval_vn + eval_vc + eval_vw) > 0 else 0
f1 = round(2* precision* recall/ (precision + recall), 2) if (precision + recall) > 0 else 0

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1: {f1}")                

Precision: 68.96
Recall: 64.13
F1: 66.46
